# [Module 3.1] Personalize 솔류션 및 솔류션 버전 생성

이 노트북은 Module1에서 생성한 데이타셋 그룹, 데이타셋을 바탕으로 아래와 같은 작업을 합니다.

* 레서피(알고리즘) 선택 및 7가지 솔류션 생성
* 7가지 솔류션 버전 생성

--- 
이 노트북은 약 50분 소요 됩니다.



In [1]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time


다음으로 여러분의 환경이 Amazon Personalize와 성공적으로 통신할 수 있는지 확인해야 합니다.

In [2]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')

아래 코드 셀은 이전 notebook에서 저장했던 공유 변수들을 불러옵니다.

In [3]:
%store -r

생성할 오브젝트의 끝에 임의의 숫자를 부여하기 위해 suffix 정의

In [4]:
# suffix = str(np.random.uniform())[4:9]

## 솔루션 및 버전 생성

Amazon Personalize에서 훈련된 모델을 솔루션이라고 하며, 각 솔루션에는 모델이 훈련되었을 때 주어진 데이터량과 관련된 많은 특정 버전들이 있을 수 있습니다.

우선, Amazon Personalize에서 지원되는 모든 레시피(레시피는 아직 데이터에 대해 훈련되지 않은 알고리즘입니다.)들을 나열합니다. 리스트업된 레시피들 중 하나를 선택하고 이를 사용하여 모델을 빌드해 보세요.

이 LAB에서 아래와 같은 모든 솔루션을 생성하여 성능을 비교하여 봅니다.

    1) User-Personalization 
    2) HRNN
    3) HRNN Coldstart
    4) HRNN-Metadata
    5) popularity-count 
    6) SIMS
    7) Personalized Ranking

이 프로세스의 완료는 실제로 40분 이상 소요됩니다. 작업이 완료될 때까지(즉, 활성화 상태가 될 때까지) while 루프를 수행하는 방법도 있지만, 이렇게 하면 다른 셀의 실행을 차단하게 됩니다. 따라서, 많은 모델을 만들어 신속하게 배포하려면 while 루프를 사용하는 대신, 필요한 솔루션 버전들을 생성 후, SageMaker 및 Cloudwatch에서 업데이트를 확인하세요.

###  레시피 리스트 확인

In [5]:
recipe_list = personalize.list_recipes()
for recipe in recipe_list['recipes']:
    print(recipe['recipeArn'])

arn:aws:personalize:::recipe/aws-hrnn
arn:aws:personalize:::recipe/aws-hrnn-coldstart
arn:aws:personalize:::recipe/aws-hrnn-metadata
arn:aws:personalize:::recipe/aws-personalized-ranking
arn:aws:personalize:::recipe/aws-popularity-count
arn:aws:personalize:::recipe/aws-sims
arn:aws:personalize:::recipe/aws-user-personalization


###  User-Personalization 솔루션 생성 

In [6]:
# Recipe 선택 
system_user_personalization_recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization"

# Solution 생성 
create_solution_response = personalize.create_solution(
    name = "Movielens-user-personalization-" + suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = system_user_personalization_recipe_arn,
)

In [7]:
user_personalization_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-user-personalization-59673",
  "ResponseMetadata": {
    "RequestId": "326ebff0-6b51-47ec-91b9-36af7b1738df",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 04 Sep 2020 11:41:34 GMT",
      "x-amzn-requestid": "326ebff0-6b51-47ec-91b9-36af7b1738df",
      "content-length": "111",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [9]:
# 솔루션 버전 생성
create_solution_version_response = personalize.create_solution_version(
    solutionArn = user_personalization_solution_arn
)

user_personalization_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-user-personalization-59673/6e16c11b",
  "ResponseMetadata": {
    "RequestId": "c42935c7-2359-4cd7-8321-9000ecffefd7",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 04 Sep 2020 11:42:20 GMT",
      "x-amzn-requestid": "c42935c7-2359-4cd7-8321-9000ecffefd7",
      "content-length": "127",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


###  Popularity Count 솔루션 생성 

In [10]:
# Recipe 선택 
system_popularity_recipe_arn = "arn:aws:personalize:::recipe/aws-popularity-count"

# Solution 생성 
create_solution_response = personalize.create_solution(
    name = "Movielens-popularity-" + suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = system_popularity_recipe_arn,
)



In [11]:
popularity_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-popularity-59673",
  "ResponseMetadata": {
    "RequestId": "cb933ec9-d9ec-4ece-9745-4eea9aa4c7e6",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 04 Sep 2020 11:42:22 GMT",
      "x-amzn-requestid": "cb933ec9-d9ec-4ece-9745-4eea9aa4c7e6",
      "content-length": "101",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [13]:
# 솔루션 버전 생성
create_solution_version_response = personalize.create_solution_version(
    solutionArn = popularity_solution_arn
)

popularity_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-popularity-59673/54f97f23",
  "ResponseMetadata": {
    "RequestId": "4bd61d5b-549a-477b-a594-875c0d2974b3",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 04 Sep 2020 11:42:43 GMT",
      "x-amzn-requestid": "4bd61d5b-549a-477b-a594-875c0d2974b3",
      "content-length": "117",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### HRNN 솔루션 생성

In [14]:
 # Recipe 선택 
system_hrnn_recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn"


# Solution 생성 
create_solution_response = personalize.create_solution(
    name = "Movielens-aws-hrnn-" + suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = system_hrnn_recipe_arn,
)

hrnn_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-aws-hrnn-59673",
  "ResponseMetadata": {
    "RequestId": "33a8fc27-1677-45eb-bfb5-89aca4e84960",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 04 Sep 2020 11:42:47 GMT",
      "x-amzn-requestid": "33a8fc27-1677-45eb-bfb5-89aca4e84960",
      "content-length": "99",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [15]:
# 솔루션 버전 생성
create_solution_version_response = personalize.create_solution_version(
    solutionArn = hrnn_solution_arn
)

hrnn_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-aws-hrnn-59673/8b63c375",
  "ResponseMetadata": {
    "RequestId": "c526e8fc-e22a-4e2c-9f31-f5b24052b587",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 04 Sep 2020 11:42:48 GMT",
      "x-amzn-requestid": "c526e8fc-e22a-4e2c-9f31-f5b24052b587",
      "content-length": "115",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### HRNN-Meta 솔루션 생성

In [16]:
 # Recipe 선택 
system_hrnn_meta_recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn-metadata"

# Solution 생성 
create_solution_response = personalize.create_solution(
    name = "Movielens-aws-hrnn-metadata-" + suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = system_hrnn_meta_recipe_arn,
)

hrnn_meta_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-aws-hrnn-metadata-59673",
  "ResponseMetadata": {
    "RequestId": "19e9a6bf-bf1d-49c5-bb21-9de547980793",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 04 Sep 2020 11:42:49 GMT",
      "x-amzn-requestid": "19e9a6bf-bf1d-49c5-bb21-9de547980793",
      "content-length": "108",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [17]:
# 솔루션 버전 생성
create_solution_version_response = personalize.create_solution_version(
    solutionArn = hrnn_meta_solution_arn
)

hrnn_meta_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-aws-hrnn-metadata-59673/e248e1ac",
  "ResponseMetadata": {
    "RequestId": "a1f85927-05e1-4194-a158-c4474a89d706",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 04 Sep 2020 11:42:49 GMT",
      "x-amzn-requestid": "a1f85927-05e1-4194-a158-c4474a89d706",
      "content-length": "124",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### HRNN ColdStart 솔루션 생성

In [18]:
 # Recipe 선택 
system_hrnn_coldstart_recipe_arn = "arn:aws:personalize:::recipe/aws-hrnn-coldstart"

# Solution 생성 
create_solution_response = personalize.create_solution(
    name = "Movielens-hrnn-coldstart-" + suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = system_hrnn_coldstart_recipe_arn,
    solutionConfig = {
        "featureTransformationParameters" : {
            'cold_start_max_duration' : '5',
            'cold_start_relative_from' : 'latestItem',
            'cold_start_max_interactions':'15'
        }
    }
)

hrnn_coldstart_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-hrnn-coldstart-59673",
  "ResponseMetadata": {
    "RequestId": "2eb80d9d-7782-422f-9505-4c9e0f3e0216",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 04 Sep 2020 11:42:50 GMT",
      "x-amzn-requestid": "2eb80d9d-7782-422f-9505-4c9e0f3e0216",
      "content-length": "105",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [19]:
# 솔루션 버전 생성
create_solution_version_response = personalize.create_solution_version(
    solutionArn = hrnn_coldstart_solution_arn
)

hrnn_coldstart_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-hrnn-coldstart-59673/1e1825b8",
  "ResponseMetadata": {
    "RequestId": "5c29a6e4-0a6c-4407-b266-42dbc67ad50a",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 04 Sep 2020 11:42:51 GMT",
      "x-amzn-requestid": "5c29a6e4-0a6c-4407-b266-42dbc67ad50a",
      "content-length": "121",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### SIMS 솔루션 생성 

In [20]:
 # Recipe 선택 
system_sims_recipe_arn = "arn:aws:personalize:::recipe/aws-sims"

# Solution 생성 
create_solution_response = personalize.create_solution(
    name = "Movielens-sims-" + suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = system_sims_recipe_arn
)

sims_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-sims-59673",
  "ResponseMetadata": {
    "RequestId": "56e3ef1d-67ed-43a6-9c0b-0b3e7dd90cb8",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 04 Sep 2020 11:42:53 GMT",
      "x-amzn-requestid": "56e3ef1d-67ed-43a6-9c0b-0b3e7dd90cb8",
      "content-length": "95",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [21]:
# 솔루션 버전 생성
create_solution_version_response = personalize.create_solution_version(
    solutionArn = sims_solution_arn
)
sims_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-sims-59673/02f182b0",
  "ResponseMetadata": {
    "RequestId": "100862a7-d2bf-46bc-b242-6267079dcdc6",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 04 Sep 2020 11:42:54 GMT",
      "x-amzn-requestid": "100862a7-d2bf-46bc-b242-6267079dcdc6",
      "content-length": "111",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Personalize Ranking 솔루션 생성 

In [22]:
 # Recipe 선택 
system_ranking_recipe_arn = "arn:aws:personalize:::recipe/aws-personalized-ranking"

# Solution 생성 
create_solution_response = personalize.create_solution(
    name = "Movielens-ranking-" + suffix,
    datasetGroupArn = dataset_group_arn,
    recipeArn = system_ranking_recipe_arn
)

ranking_solution_arn = create_solution_response['solutionArn']
print(json.dumps(create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-ranking-59673",
  "ResponseMetadata": {
    "RequestId": "1a7f848d-4cfc-4939-97d7-b3e27d975493",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 04 Sep 2020 11:42:57 GMT",
      "x-amzn-requestid": "1a7f848d-4cfc-4939-97d7-b3e27d975493",
      "content-length": "98",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


In [23]:
# 솔루션 버전 생성
create_solution_version_response = personalize.create_solution_version(
    solutionArn = ranking_solution_arn
)
ranking_solution_version_arn = create_solution_version_response['solutionVersionArn']
print(json.dumps(create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:057716757052:solution/Movielens-ranking-59673/aeda5dcc",
  "ResponseMetadata": {
    "RequestId": "0075d049-08d7-4948-8cd8-fddd52cb7806",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Fri, 04 Sep 2020 11:42:57 GMT",
      "x-amzn-requestid": "0075d049-08d7-4948-8cd8-fddd52cb7806",
      "content-length": "114",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### 모델 생성 확인 하기 
아래 코드 실행하여 모델 생성 여부를 확인합니다. 

In [ ]:
%%time


max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    ##user_personalization Status
    #hrnn status
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = user_personalization_solution_version_arn
    )  
    status_user_per = describe_solution_version_response["solutionVersion"]["status"]
    print("user_personalization SolutionVersion: {}".format(status_user_per))
    
    ##Popularity Status
    #hrnn status
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = popularity_solution_version_arn
    )  
    status_pop = describe_solution_version_response["solutionVersion"]["status"]
    print("Popularity SolutionVersion: {}".format(status_pop))
    
    #hrnn status
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = hrnn_solution_version_arn
    )  
    status_hrnn = describe_solution_version_response["solutionVersion"]["status"]
    print("HRNN SolutionVersion: {}".format(status_hrnn))

    #hrnn_meta status
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = hrnn_meta_solution_version_arn
    )
    status_hrnn_meta = describe_solution_version_response["solutionVersion"]["status"]    
    print("HRNN Meta SolutionVersion: {}".format(status_hrnn_meta))
    
    
    #hrnn_coldstart status
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = hrnn_coldstart_solution_version_arn
    )
    status_hrnn_cs = describe_solution_version_response["solutionVersion"]["status"]    
    print("HRNN ColdStart SolutionVersion: {}".format(status_hrnn_cs))
    
    #sims status
    describe_solution_version_response = personalize.describe_solution_version(
    solutionVersionArn = sims_solution_version_arn
    )
    status_sims = describe_solution_version_response["solutionVersion"]["status"]
    print("Sims SolutionVersion: {}".format(status_sims))
    
    #ranking status
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = ranking_solution_version_arn
    )
    status_ranking= describe_solution_version_response["solutionVersion"]["status"]    
    print("Ranking SolutionVersion: {}".format(status_ranking))    
    
    if (status_user_per == "ACTIVE" or status_user_per == "CREATE FAILED") &\
       (status_pop == "ACTIVE" or status_pop == "CREATE FAILED") &\
       (status_hrnn == "ACTIVE" or status_hrnn == "CREATE FAILED") &\
       (status_hrnn_meta == "ACTIVE" or status_hrnn_meta == "CREATE FAILED") &\
       (status_hrnn_cs == "ACTIVE" or status_hrnn_cs == "CREATE FAILED") &\
       (status_sims == "ACTIVE" or status_sims == "CREATE FAILED") &\
       (status_ranking == "ACTIVE" or status_ranking == "CREATE FAILED"):
        break
    print("-------------------------------------->")
    time.sleep(60)

print("All solution creation completed")

user_personalization SolutionVersion: CREATE IN_PROGRESS
Popularity SolutionVersion: CREATE IN_PROGRESS
HRNN SolutionVersion: CREATE IN_PROGRESS
HRNN Meta SolutionVersion: CREATE IN_PROGRESS
HRNN ColdStart SolutionVersion: CREATE IN_PROGRESS
Sims SolutionVersion: CREATE IN_PROGRESS
Ranking SolutionVersion: CREATE IN_PROGRESS
-------------------------------------->
user_personalization SolutionVersion: CREATE IN_PROGRESS
Popularity SolutionVersion: CREATE IN_PROGRESS
HRNN SolutionVersion: CREATE IN_PROGRESS
HRNN Meta SolutionVersion: CREATE IN_PROGRESS
HRNN ColdStart SolutionVersion: CREATE IN_PROGRESS
Sims SolutionVersion: CREATE IN_PROGRESS
Ranking SolutionVersion: CREATE IN_PROGRESS
-------------------------------------->
user_personalization SolutionVersion: CREATE IN_PROGRESS
Popularity SolutionVersion: CREATE IN_PROGRESS
HRNN SolutionVersion: CREATE IN_PROGRESS
HRNN Meta SolutionVersion: CREATE IN_PROGRESS
HRNN ColdStart SolutionVersion: CREATE IN_PROGRESS
Sims SolutionVersion: CR

### Store variables

In [ ]:

%store user_personalization_solution_arn
%store user_personalization_solution_version_arn

%store popularity_solution_arn
%store popularity_solution_version_arn

%store hrnn_solution_version_arn
%store hrnn_solution_arn

%store hrnn_meta_solution_version_arn
%store hrnn_meta_solution_arn


%store hrnn_coldstart_solution_version_arn
%store hrnn_coldstart_solution_arn

%store sims_solution_version_arn
%store sims_solution_arn

%store ranking_solution_version_arn
%store ranking_solution_arn
